In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)

In [2]:
df = pd.read_csv('MLB-GameLogs-1871_2016.csv', low_memory = False)
df

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,19460416,0,Tue,SLA,AL,1,DET,AL,1,1,...,Pinky Higgins,5.0,richp101,Paul Richards,2.0,newhh101,Hal Newhouser,1.0,NaN,Y
1,19460416,0,Tue,NYA,AL,1,PHA,AL,1,5,...,George Kell,5.0,handg101,Gene Handley,4.0,chrir101,Russ Christopher,1.0,NaN,Y
2,19460416,0,Tue,BOS,AL,1,WS1,AL,1,6,...,Jerry Priddy,4.0,evana101,Al Evans,2.0,wolfr102,Roger Wolff,1.0,NaN,Y
3,19460416,0,Tue,CLE,AL,1,CHA,AL,1,1,...,Thurman Tucker,8.0,tresm101,Mike Tresh,2.0,dietb101,Bill Dietrich,1.0,NaN,Y
4,19460416,0,Tue,BRO,NL,1,BSN,NL,1,3,...,Skippy Roberge,5.0,culld101,Dick Culler,6.0,sainj101,Johnny Sain,1.0,NaN,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94927,20161002,0,Sun,MIL,NL,162,COL,NL,162,6,...,Pat Valaika,4.0,adamc001,Cristhian Adames,6.0,marqg001,German Marquez,1.0,NaN,Y
94928,20161002,0,Sun,NYN,NL,162,PHI,NL,162,2,...,Andres Blanco,6.0,altha001,Aaron Altherr,9.0,eickj001,Jerad Eickhoff,1.0,NaN,Y
94929,20161002,0,Sun,LAN,NL,162,SFN,NL,162,1,...,Joe Panik,4.0,gillc001,Conor Gillaspie,5.0,moorm003,Matt Moore,1.0,NaN,Y
94930,20161002,0,Sun,PIT,NL,162,SLN,NL,162,4,...,Jedd Gyorko,4.0,gricr001,Randal Grichuk,8.0,waina001,Adam Wainwright,1.0,NaN,Y


The dataset contains a rather exhaustive list of metrics per MLB games. However, for simplification purposes we are going to do a bit of cleaning and alter this data. The following chapter explains what steps were taken during this process.

# Data Cleaning & Exploration

In [4]:
# Dropping redundant columns
dropping = []
for column in df.columns[7:]:
    if column.endswith('_def_pos'):
        dropping.append(column)
    elif '_name' in column:
        dropping.append(column)
    elif '_info' in column:
        dropping.append(column)
    elif 'lf_umpire_id' in column:
        dropping.append(column)
    elif 'rf_umpire_id' in column:
        dropping.append(column)
df = df.drop(columns = dropping)

# Removing the 2 forfeited games & column
df = df[df['forefeit'].isna()==True]
df = df.drop(columns = 'forefeit')

# Removing interrupted games & completion column
df = df[df['completion'].isna()==True]
df = df.drop(columns = 'completion')

# Remove protested games & column
df = df[df['protest'].isna()==True]
df = df.drop(columns = 'protest')

# Remove multi-headed games
df = df[df['number_of_game'] == 0]
df = df.drop(columns = 'number_of_game')

# Remove draws
df = df[df['h_score'] != df['v_score']]

The following has been done to accomodate the missing values in Day/Night, in attendance and length_minutes:

In [5]:
# Random Weighted Imputing for day_night column
choice = sorted(df[df.day_night.isna() == False]['day_night'].unique())
prob = df[df.day_night != 0]['day_night'].value_counts(normalize=True).sort_index()
df['day_night'] = df.day_night.map(lambda x: np.random.choice(choice, 1, p = prob)[0] if x not in choice else x)

# Median Imputing for attendance & length_minutes missing values
df['attendance'] = df['attendance'].fillna(value= df['attendance'].median())
df['length_minutes'] = df['length_minutes'].fillna(value= df['length_minutes'].median())

In [6]:
# Reset Index & Assign Match ID
df.reset_index(inplace = True)
df.rename(columns = {'index': 'matchID'}, inplace = True)

More columns have missing values:

In [7]:
# Check for missing values
for column in df.columns:
    if df[column].isna().sum() != 0:
        print (f"Missing in {column}: {df[column].isna().sum()}")

Missing in 2b_umpire_id: 4872
Missing in 3b_umpire_id: 128
Missing in saving_pitcher_id: 46722
Missing in winning_rbi_batter_id: 5519


We are not going to fill these in as we are going to drop those _id columns later in the process.

## Dataset reformatting

As we have seen above, our dataset consists of one observation per game. However, considering our business case, we need to convert this 'one-observation per game' to a winner/loser dataset.
* This means we need to create a column that will hold the winning/losing status of a team (during a certain game).
* It also means we are going to double the number of observations (there is 1 winner and 1 loser per game).

This dataset is going to be our base dataset. We will take more informed decisions later on during the [analysis](./index.ipynb) but we will end this notebook by saving the dataset as a `.csv` file and the alteration to the data will be explained when made.

In [8]:
# Categorize winning team for restructuring dataset
winning_team = []
for i in range(0,len(df)):
    if df['h_score'][i] > df['v_score'][i] == True:
        winning_team.append('Home')
    else:
        winning_team.append('Visitor')
df['winning_team'] = winning_team

# Define unique home/visitor stats
h_stats = []
for column in df.columns:
    if str(column).startswith('h_'):
        h_stats.append(column)

v_stats = []
for column in df.columns:
    if str(column).startswith('v_'):
        v_stats.append(column)

In [9]:
# Extracting only winning team and respective statistics
dfwin1 = df[df['winning_team'] == 'Home']
dfwin1 = dfwin1.drop(columns = v_stats, axis = 1)

dfwin2 = df[df['winning_team'] == 'Visitor']
dfwin2 = dfwin2.drop(columns = h_stats, axis = 1)

# Renaming h_ and v_ columns
for column in dfwin1.columns:
    if str(column).startswith('h_') == True:
        dfwin1.rename(columns = {column: column[2:]}, inplace = True)
        
for column in dfwin2.columns:
    if str(column).startswith('v_') == True:
        dfwin2.rename(columns = {column: column[2:]}, inplace = True)

# Merging both win df
dfwin = pd.concat([dfwin1, dfwin2], sort = False) 

# Assigning boolean target variable "winner"
dfwin["winner"] = np.nan
dfwin['winner'].fillna(value = 1, inplace = True)

In [10]:
# Extracting only losing team and respective statistics
dfloss1 = df[df['winning_team'] == 'Home']
dfloss1 = dfloss1.drop(columns = h_stats, axis = 1)

dfloss2 = df[df['winning_team'] == 'Visitor']
dfloss2 = dfloss2.drop(columns = v_stats, axis = 1)

# Renaming h_ and v_ columns
for column in dfloss1.columns:
    if str(column).startswith('v_') == True:
        dfloss1.rename(columns = {column: column[2:]}, inplace = True)
        
for column in dfloss2.columns:
    if str(column).startswith('h_') == True:
        dfloss2.rename(columns = {column: column[2:]}, inplace = True)

# Merging both loss df
dfloss = pd.concat([dfloss1, dfloss2], sort = False)

# Assigning boolean target variable "winner"
dfloss["winner"] = np.nan
dfloss['winner'].fillna(value = 0, inplace = True)

In [11]:
# Combine both win and loss dataframe
df = pd.concat([dfwin, dfloss], sort = False)

In [12]:
df.to_csv('df.csv')

The analysis takes palce in the following notebook: [index.ipynb](./index.ipynb)